In [31]:
# std
import os
from os import path
import time

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
from src import helpers, configuration, filtering, bandpower, plots, statistics as stats
from src.configuration import cfg
os.chdir('analysis-adelie')

# statistics
from scipy.stats import ttest_ind

import mne
from mne import viz

# plots
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib

font = {'family':'DejaVu Sans', 'weight':'normal', 'size':15}
matplotlib.rc('font', **font)

%matplotlib qt

### General

In [32]:
def get_channelsList(config, subject='adelie'):
    subject_paths = helpers.get_config_value(config, 'paths', 'subjects', subject)
    base_path = helpers.get_config_value(config, 'paths', 'base')
    file_path = f"{base_path}/{subject_paths['prefix']}/offline/fif/{subject_paths['channels_path']}"
    with open(file_path, 'r') as channels_file:
        all_channels = channels_file.read().strip()
    return [channel for channel in all_channels.split('\n') if channel not in config['columns_to_remove']]

events = None
def load_raw_mne_from_fif(data_type, subject='adelie', recording=0, montage='standard_1020', config=cfg):
    global events
    """loads the data and returns an instance of mne.Raw
    
    Parameters
    ----------
    data_type : string
      type of the data, right now two options are valid: `baseline` or `meditation`
    subject: string
      name of the subject
    recording: int
      number of recording, if you have multiple of same type and subject
    montage: string
      the type of montage that was used for the recording see: https://mne.tools/dev/generated/mne.channels.make_standard_montage.html
      
    Returns
    -------
    a mne.Raw instance that has the correct montage and info and is ready to be plotted
    """
    subject_paths = helpers.get_config_value(config, 'paths', 'subjects', subject)
    base_path = helpers.get_config_value(config, 'paths', 'base')
    recording_id = helpers.get_config_value(subject_paths, 'recordings', data_type)[recording]
    file_path = f"{base_path}{subject_paths['prefix']}/offline/fif/{recording_id}-raw.fif"
    
    # Create a digitization of the montage
    digitization = mne.channels.make_standard_montage(montage)
    channels = get_channelsList(config, subject=subject)
    
    # Read from fif file
    raw = mne.io.read_raw_fif(file_path, preload=True)
    events = mne.find_events(raw)

    # Create info with some useful information
    info = mne.create_info(get_channelsList(config), sfreq=config['sampling_frequency'], ch_types='eeg')
    raw.info = info
    
    # set the montage
    raw.set_montage(digitization)
    
    raw = raw.pick_types(eeg=True, stim=True)
    raw.set_eeg_reference('average', projection=True)#.apply_proj()
    
    return raw

In [33]:
def normalise_layout_pos(layout):
    _t = layout.pos[:, :2]
    _t -= np.mean(_t, 0)
    _t /= np.max(_t, 0)
    
    return _t

In [257]:
signals = load_raw_mne_from_fif('baseline', subject='sam', config=cfg)
layout = mne.channels.make_eeg_layout(signals.info)
normalised_pos = normalise_layout_pos(layout)


# for egi:
egi_electrodes = pd.read_csv('ratio_theta_mr.csv').drop(columns=['Unnamed: 0']).columns

montage_egi = mne.channels.make_standard_montage('EGI_256')
info_egi = mne.create_info(montage.ch_names, cfg_mr['sampling_frequency'], 'eeg').set_montage(montage_egi)
info_egi['bads'] = set(montage.ch_names) - set(egi_electrodes)
layout_egi = mne.channels.make_eeg_layout(info_egi)
normalised_pos_egi = normalise_layout_pos(layout_egi)

Opening raw data file C:/Users/adeli/OneDrive/Documents/GitHub/data/AlphaTheta/sam-AlphaTheta/offline/fif/baseline11-raw.fif...
Isotrak not found
    Range : 0 ... 38030 =      0.000 ...   126.767 secs
Ready.
Reading 0 ... 38030  =      0.000 ...   126.767 secs...
2 events found
Event IDs: [15 16]
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


In [271]:
def get_mne_maps(csv, vmin, vmax, title="", mr=False):
    data = pd.read_csv(csv).fillna(0)
    data = data.drop(columns=['Unnamed: 0'])

    time_idx_to_plot = np.linspace(0, data.shape[0] - 1, num=1, dtype='int')
    elec = data.columns

    frames = []
    fig, ax = plt.subplots()
    cNorm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    sm = cm.ScalarMappable(norm=cNorm, cmap=cmap)

    viz.plot_topomap(data.loc[0].values,
                     cmap='RdBu', vmin=vmin, vmax=vmax,
                     sphere=1.,
                     pos=normalised_pos_egi if mr else normalised_pos,
                     sensors=False,
                     show_names=True,
                     show=False,
                     names=elec)

    plt.title(title)

    fig.colorbar(sm, ax=ax, orientation='vertical')
    plt.tight_layout()

    # this has to be the last thing
    fig.canvas.draw()

    mat = np.array(fig.canvas.renderer._renderer)
    frames.append(mat)
    plt.show()

In [36]:
cmap='RdBu'

In [313]:
subjects= ['adelie', 'sam', 'sam2', 'sam3', 'sam4', 'rap', 'rap2', 'rap3', 'arn', 'mr']
id_to_name = {'adelie': "Adélie", 'sam': "Sam", 'sam2': "Sam2", 'sam3': "Sam3", 'sam4': "Sam4", 'rap': "Raphaël", 'rap2': "Raphaël2", 'rap3': "Raphaël3", 'arn': "Arnaud", 'mr': 'Matthieu Ricard'}

In [332]:
def plot_all_figures(all_data, vmin, vmax, subtitle):
    frames = []
    n_subplots = len(all_data)
    fig, axs = plt.subplots(nrows=2, ncols=int(np.ceil(n_subplots/2)), figsize=(5, 5))

    cNorm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    sm = cm.ScalarMappable(norm=cNorm, cmap=cmap)

    flattened_axs = [item for sublist in axs for item in sublist]

    for ax in flattened_axs:
        for plt_idx, subject in enumerate(all_data.items()):
            viz.plot_topomap(subject[1].fillna(0).loc[0].values,
                             cmap='RdBu', vmin=vmin, vmax=vmax,
                             sphere=1.,
                             pos=normalised_pos_egi if subject[0] == 'mr' else normalised_pos,
                             axes=flattened_axs[plt_idx],
                             sensors=False,
                             show_names=subject[0] != 'mr',
                             show=False,
                             names=subject[1].columns)
            flattened_axs[plt_idx].set_title("{}".format(id_to_name[subject[0]]), fontsize=14)

    plt.suptitle(subtitle, y=1)

    fig.colorbar(sm, ax=axs, orientation='vertical')
    # this has to be the last thing
    fig.canvas.draw()

    mat = np.array(fig.canvas.renderer._renderer)
    frames.append(mat)
    plt.show()

### Predefined metrics: ratio meditation/baseline

#### Theta

In [324]:
# set the same color scale for all subjects
# this scale depends on the min and max values over all subjects
# this log scale is centered on zero

data_t = {subject: pd.read_csv('ratio_theta_{}.csv'.format(subject)).drop(columns=['Unnamed: 0']) for subject in subjects}

vmin_t = min(map(lambda x: min(x.loc[0].values), data_t.values()))
vmax_t = max(map(lambda x: max(x.loc[0].values), data_t.values()))

abs_max = max(abs(vmin_t), abs(vmax_t))
vmin_t, vmax_t = -abs_max, abs_max

In [299]:
data_mr_t = get_mne_maps('ratio_theta_mr.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Matthieu Ricard", mr=True)

In [180]:
data_ad_t = get_mne_maps('ratio_theta_adelie.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Adélie")

In [180]:
data_sam_t = get_mne_maps('ratio_theta_sam.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Sam")

In [189]:
data_sam2_t = get_mne_maps('ratio_theta_sam2.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Sam2")

In [180]:
data_sam3_t = get_mne_maps('ratio_theta_sam3.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Sam3")

In [304]:
data_sam4_t = get_mne_maps('ratio_theta_sam4.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Sam4")

In [316]:
data_rap_t = get_mne_maps('ratio_theta_rap.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Raphaël")

In [180]:
data_rap2_t = get_mne_maps('ratio_theta_rap2.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Raphaël2")

In [180]:
data_rap3_t = get_mne_maps('ratio_theta_rap3.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Raphaël3")

In [269]:
data_arn_t = get_mne_maps('ratio_theta_arn.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Arnaud")

In [315]:
data_mr_t = get_mne_maps('ratio_theta_mr.csv', vmin_t, vmax_t, title="Ratio Meditation/Baseline for Theta band | Matthieu Ricardos", mr=True)

CONCLUSION: all figures in one

In [341]:
plot_all_figures(data_t, vmin_t, vmax_t,'Ratio Meditation/Baseline for Theta band')

#### Delta

In [286]:
# set the same color scale for all subjects
# this scale depends on the min and max values over all subjects
# this log scale is centered on zero

data_d = {subject: pd.read_csv('ratio_delta_{}.csv'.format(subject)).drop(columns=['Unnamed: 0']) for subject in subjects}

vmin_d = min(map(lambda x: min(x.loc[0].values), data_d.values()))
vmax_d = max(map(lambda x: max(x.loc[0].values), data_d.values()))

abs_max = max(abs(vmin_d), abs(vmax_d))
vmin_d, vmax_d = -abs_max, abs_max

In [298]:
data_mr_d = get_mne_maps('ratio_delta_mr.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Matthieu Ricard", mr=True)

In [180]:
data_ad_d = get_mne_maps('ratio_delta_adelie.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Adélie")

In [306]:
data_sam_d = get_mne_maps('ratio_delta_sam.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Sam")

In [86]:
data_sam2_d = get_mne_maps('ratio_delta_sam2.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Sam2")

In [86]:
data_sam3_d = get_mne_maps('ratio_delta_sam3.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Sam3")

In [86]:
data_sam4_d = get_mne_maps('ratio_delta_sam4.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Sam4")

In [86]:
data_rap_d = get_mne_maps('ratio_delta_rap.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Raphaël")

In [86]:
data_rap2_d = get_mne_maps('ratio_delta_rap2.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Raphaël2")

In [86]:
data_rap3_d = get_mne_maps('ratio_delta_rap3.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Raphaël3")

In [86]:
data_arn_d = get_mne_maps('ratio_delta_adelie.csv', vmin_d, vmax_d, title="Ratio Meditation/Baseline for Delta band | Arnaud")

CONCLUSION: all figures in one

In [340]:
plot_all_figures(data_d, vmin_d, vmax_d, 'Ratio Meditation/Baseline for Delta band')

#### Gamma

In [334]:
# set the same color scale for all subjects
# this scale depends on the min and max values over all subjects
# this log scale is centered on zero

data_g = {subject: pd.read_csv('ratio_gamma_{}.csv'.format(subject)).drop(columns=['Unnamed: 0']) for subject in subjects}

vmin_g = min(map(lambda x: min(x.loc[0].values), data_g.values()))
vmax_g = max(map(lambda x: max(x.loc[0].values), data_g.values()))

abs_max = max(abs(vmin_g), abs(vmax_g))
vmin_g, vmax_g = -abs_max, abs_max

In [297]:
data_mr_g = get_mne_maps('ratio_gamma_mr.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Matthieu Ricard", mr=True)

In [305]:
data_ad_g = get_mne_maps('ratio_gamma_adelie.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Adélie")

In [114]:
data_sam_g = get_mne_maps('ratio_gamma_sam.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Sam")

In [116]:
data_sam2_g = get_mne_maps('ratio_gamma_sam2.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Sam2")

In [117]:
data_sam3_g = get_mne_maps('ratio_gamma_sam3.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Sam3")

In [118]:
data_sam4_g = get_mne_maps('ratio_gamma_sam4.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Sam4")

In [119]:
data_rap_g = get_mne_maps('ratio_gamma_rap.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Raphaël")

In [120]:
data_rap2_g = get_mne_maps('ratio_gamma_rap2.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Raphaël2")

In [121]:
data_rap3_g = get_mne_maps('ratio_gamma_rap3.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Raphaël3")

In [122]:
data_arn_g = get_mne_maps('ratio_gamma_arn.csv', vmin_g, vmax_g, title="Ratio Meditation/Baseline for Gamma band | Arnaud")

CONCLUSION: all figures in one

In [339]:
plot_all_figures(data_g, vmin_g, vmax_g, 'Ratio Meditation/Baseline for Gamma band')

#### Alpha / Theta

In [336]:
# set the same color scale for all subjects
# this scale depends on the min and max values over all subjects
# this log scale is centered on zero

data_at = {subject: pd.read_csv('ratio_at_{}.csv'.format(subject)).drop(columns=['Unnamed: 0']) for subject in subjects}

vmin_at = min(map(lambda x: min(x.loc[0].values), data_at.values()))
vmax_at = max(map(lambda x: max(x.loc[0].values), data_at.values()))

abs_max = max(abs(vmin_at), abs(vmax_at))
vmin_at, vmax_at = -abs_max, abs_max

In [303]:
data_mr_at = get_mne_maps('ratio_at_mr.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta \n band | Matthieu Ricard", mr=True)

In [161]:
data_ad_at = get_mne_maps('ratio_at_adelie.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Adélie")

In [151]:
data_sam_at = get_mne_maps('ratio_at_sam.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Sam")

In [137]:
data_sam2_at = get_mne_maps('ratio_at_sam2.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Sam2")

In [138]:
data_sam3_at = get_mne_maps('ratio_at_sam3.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Sam3")

In [342]:
data_sam4_at = get_mne_maps('ratio_at_sam4.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Sam4")

In [140]:
data_rap_at = get_mne_maps('ratio_at_rap.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Raphaël")

In [141]:
data_rap2_at = get_mne_maps('ratio_at_rap2.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Raphaël2")

In [142]:
data_rap3_at = get_mne_maps('ratio_at_rap3.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Raphaël3")

In [122]:
data_arn_at = get_mne_maps('ratio_at_arn.csv', vmin_at, vmax_at, title="Ratio Meditation/Baseline for Alpha/Theta band | Arnaud")

CONCLUSION: all figures in one

In [338]:
plot_all_figures(data_at, vmin_at, vmax_at, 'Ratio Meditation/Baseline for Alpha/Theta band')